<a href="https://colab.research.google.com/github/mjn6862/Speed_Challenge/blob/master/Matthew_categorical_Speed_Challenge_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Comma AI Speed Challenge**

  This notebook will contain (hopefully) all of the functions you need to import the data into your model.

  ***Be sure to train with GPU acceleration enabled***

**Import Statements**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
import os

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

**Custom Data Generator**

This works (I think) for giving two sequential images to a Keras Functional model as well as the velocity associated with the second image.

At this point, don't worry about how this works. If you need something changed or fixed, just ask. This is the boring part anyways.

In [ ]:
# max speed is 28.130404
# bucket speed into 15 2-mph bins for categorical data

In [ ]:
# need to mount drive before using this
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True, categorical=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.categorical = categorical # my edition
        self.on_epoch_end()
        self.direct = "./drive/My Drive/commai_dataset/"

    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(np.floor(len(self.list_IDs) / self.batch_size))
        return len(self.list_IDs)
        
    def __getitem__(self, index):
        'Generate one batch of data'
        
        # Find list of IDs
        list_IDs_temp = self.list_IDs[index]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.load(self.direct+"data/data_" + list_IDs_temp +".npy")
        # normalize: my edition
        X = X/255
        x1 = X[0:101,:,:,:]
        x2 = X[1:102,:,:,:]
        y = np.load(self.direct+"labels/label_" + list_IDs_temp +".npy")
        y = y[1:]
        if self.categorical == True: # my edition
          y = np.array([spd//2 for spd in y],dtype=int)

        return [x1, x2], y

**Define custom loss function**

This is not well tested, neither is it optimized. You might not even want to use this function.

Keras backend functions are a powerful tool for writing custom loss functions. To define a loss function it just has to accept *y_true* and *y_pred* as arguments and return a float.

To use your new loss function, change the argument in *model.compile()*.

In [ ]:
def sum_sq_err(y_true, y_pred):
    return tf.keras.backend.sum(tf.keras.backend.square(y_true - y_pred))

**Define the test-train split and create the Data Generator**

In [ ]:
params = {'dim': (110,320),
          'batch_size': 101,
          'n_classes': 1,
          'n_channels': 3,
          'shuffle': False,
          'categorical': True}

train_data = []
train_label = []
valid_data = []
valid_label = []

for i in range(70):
    train_data.append("%03d" %i)
    train_data.append("%03d" %(i+100))

for i in range(70, 100):
    valid_data.append("%03d" %i)
    valid_data.append("%03d" %(i+100))



partition={'train':train_data, 'validation':valid_data}
labels = {'train': train_label,'validation':valid_label}


cat_train_generator = DataGenerator(partition['train'], labels['train'], **params)
cat_val_generator = DataGenerator(partition['validation'], labels['validation'], **params)

**Define the input layers**

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, concatenate, Dropout

In [ ]:
inputA = Input(shape=(110,320,3), name='first_image') 
inputB = Input(shape=(110,320,3), name='second_image') 

**Define the model**



In [ ]:
convA = Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu', name='convA')(inputA)
dropA = Dropout(0.1, name='dropA')(convA)

convB = Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu', name='convB')(inputB)
dropB = Dropout(0.1, name='dropB')(convB)

conc = concatenate(inputs=[dropA,dropB],name='conc')
conv1 = Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu', name='conv1')(conc)
drop1 = Dropout(0.1,name='drop1')(conv1)
conv2 = Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu', name='conv2')(drop1)
drop2 = Dropout(0.1, name='drop2')(conv2)
#conv3 = Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu', name='conv3')(conv2)

flat = Flatten(name='flat')(drop2)

fc1 = Dense(128, activation='relu', name='fc1')(flat)
drop3 = Dropout(0.1, name='drop3')(fc1)

fc2 = Dense(128, activation='relu', name='fc2')(drop3)
drop4 = Dropout(0.1, name='drop4')(fc2)

fc3 = Dense(128, activation='relu', name='fc3')(drop4)
drop5 = Dropout(0.1, name='drop5')(fc3)

out = Dense(15, name='out')(drop5)

In [ ]:
cat_model = tf.keras.Model(inputs=[inputA,inputB], outputs=out)

In [ ]:
cat_model.summary()

**Declare the optimizer and loss function, then compile your *less ridiculous*  model**

In [ ]:
cat_model.compile(optimizer=tf.keras.optimizers.Adam(),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

**Prepare the TensorBoard and Callbacks**

In [ ]:
logdir = os.path.join("speed_logs2", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)

earlystop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

keep = 30 # how many epochs to keep 0.001 as the learning rate
def scheduler(epoch):
  if epoch < keep:
    return 0.0005
  else:
    return 0.0005*tf.math.exp(0.1*(keep-epoch))

schedule_callback = keras.callbacks.LearningRateScheduler(scheduler)

**Train using the fit_generator**

In [ ]:
cat_model.fit_generator(generator=cat_train_generator,
                    validation_data=cat_val_generator,
                    verbose=1, 
                    epochs=30,  
          callbacks=[tensorboard_callback, 
                               earlystop_callback])

In [ ]:
%tensorboard --logdir \speed_logs2

**Visualize the predictions**


In [ ]:
predictions = cat_model.predict(cat_val_generator)

In [ ]:
# convert outputs from logits to probability using softmax
smax = tf.keras.layers.Softmax(name='smax')(out)
prob_model = tf.keras.Model(inputs=[inputA,inputB],outputs=smax)

In [ ]:
# get softmax predictions
predictions = prob_model.predict(cat_val_generator)

In [ ]:
# convert to categorical predictions
cat_preds = np.array([np.argmax(pred) for pred in predictions])

In [ ]:
# get all validation labels from the generator into one array
cat_val_labels = np.zeros(cat_preds.shape, dtype=int)
for i in range(60):
  cat_val_labels[i*101:(i+1)*101] = cat_val_generator[i][1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# get a list containing arrays of predictions, one array for each class
sorted_predictions = []
for i in range(15):
  preds = cat_preds[np.where(cat_val_labels==i)]
  sorted_predictions.append(preds)

In [ ]:
sort_bins = np.arange(0,30,1) # max label is 26.05
pred_bins = np.arange(0,int(max(cat_preds))+1,1)
label_bins = np.arange(0,2*int(max(cat_preds))+2,2)

In [ ]:
# use histograms to see how well each size mph bin was predicted
num_rows = 8
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(4*2*num_cols, 4*num_rows))
for i in range(len(sorted_predictions)-1):
  plt.subplot(num_rows,2*num_cols, 2*i+1)
  plt.hist(sorted_predictions[i], bins = pred_bins)
  plt.xticks(ticks=pred_bins,labels=label_bins, rotation=-30)
  plt.title('{}-{} mph'.format(2*sort_bins[i],2*sort_bins[i+1]))
plt.tight_layout()
plt.show()

In [ ]:
prediction_model.compile(optimizer=keras.optimizers.Adam(),
               loss='mean_squared_error')

In [ ]:
prediction_model.evaluate(validation_generator)